# Tuning the kinetic parameters for asphaltene precipitation and aggregation

This code calculates kp and kag coefficients to fit the differential equation system to experimental data. From the best calculated kp and kag, a0, a1 and c0 coefficients are calculated. These coefficients depend on Temperature and viscosity.

In the case of a Python script file (`*.py`), I will write a [module-level docstring][mod_docstring] (because you can't write markdown in a Python script).

<!--refs-->
[mod_docstring]: https://realpython.com/documenting-python-code/

In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

In [ ]:
# read excel data
file_name = 'Ag_kinetics.xlsx'
df = pd.read_excel(file_name, sheet_name='Info', header=7, usecols='b:q')
df = df[df['sheet'].notna()]
display(df)

In [ ]:
# pass raw data to numpy arrays
data = df.to_numpy()
ceq = data[:,9]
mu = data[:,10]
ca0 = data[:,15]
sheet_names = data[:,1]

In [35]:
t_exp = []
c_exp = []

# loop over the sheet names and read each sheet into a separate DataFrame
for sheet_name in sheet_names:
    # read the relevant columns from the sheet into a DataFrame
    df = pd.read_excel(file_name, sheet_name=sheet_name, usecols='b:c')
    df = df.to_numpy()
    t_exp.append(df[:,0])
    c_exp.append(df[:,1])

In [ ]:
# define functions
# NOTE: you don't need to write docstrings for every function (though you could), but write enough so that it is clear how to use your functions
def derivatives(t, y, kp, kag):
    '''
    arguments
    ---------
    t : float
        time [min]
    y : array[float]
        xyz
    kP : float
        precipitation kinetics [1/s]
    kAg : float
        aggregation kinetics [1/s]
    
    return
    ------
    '''
    cf, cag, c = y
    
    dcfdt = -kp*(cf - ceq)
    dcagdt = kag*c**2
    dcdt = kp*(cf - ceq) - kag*c**2
    
    return [dcfdt, dcagdt, dcdt]

# time span
t_span = (0.0, t_exp[-1])

# initial conditions
y0 = [1.0, 0.0, 0.0]

# Define a function that returns the model values for given parameters
def model(t, kp, kag):
    sol = solve_ivp(fun=lambda t, y: derivatives(t, y, kp, kag), t_span=t_span, y0=y0, t_eval=t)
    c_model = sol.y[0]
    return c_model

Use curve_fit to find the best coefficient values to fit experimental data

In [ ]:
#define initial guesses
p0 = [.5, .5]

# Use curve_fit to find the best values for kp and kag
popt, pcov = curve_fit(model, t_exp[:,1], c_exp[:,1], p0=p0, method='dogbox')

# Extract the best values for kp and kag
kp_best, kag_best = popt

# Print the best values
print(f"Best kp: {kp_best}")
print(f"Best kag: {kag_best}")

In [ ]:
# Plot the model predictions using the best coefficients
sol_best = solve_ivp(fun=lambda t, y: derivatives(t, y, kp_best, kag_best), t_span=t_span, y0=y0, t_eval=t_exp)
plt.plot(t_exp, c_exp, label='exp')
#plt.plot(sol_best.t, sol_best.y[0], label='cf')
plt.plot(sol_best.t, sol_best.y[1], label='cag')
#plt.plot(sol_best.t, sol_best.y[2], label='c')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.show()

#Show R2
print('R2 Score:',r2_score(c_exp,sol_best.y[1])*100,'%')

Calculating a0, a1 and c0, from the obtained best kp and kag. The coefficients depend on T and mu.

In [ ]:
# Define the functions for kp and kag
def kp(T, a0, a1):
    return a0 / np.exp(a1 / T)

def kag(T, mu, ca0, c0):
    R = 8.314 # J/mol.K, gas constant
    return ca0 * ((R * T) / (mu * 1000)) * c0

# Define the data points
T = np.array([313, 313, 313, 353, 353, 353])
best_kp_set = np.array([1.9981, 4.9979, 8.0007, 4.9997, 4.9739, 2.7972])
best_kag_set = np.array([0.4909, 0.7161, 0.2528, 0.3990, 0.3091, 0.2873])
ca0 = np.array([14.7525,13.5699,13.8464,12.4669,12.7609,13.2840])

# Define the function to fit
def fit_func(data, a0, a1, c0):
    T, mu = data
    k = np.concatenate((kp(T, a0, a1), kag(T, mu, ca0, c0)))
    return k

# Define the data as a tuple
data = (T, mu)

# Perform the fit
p0 = np.array([.1, .1, .1]) # initial guesses for a0, a1, and c0
popt, pcov = curve_fit(fit_func, data, np.concatenate((best_kp_set, best_kag_set)), p0=p0)

# Extract parameters
a0, a1, c0 = popt

# Print the results
print(f"a0 = {a0:.4f}")
print(f"a1 = {a1:.4f}")
print(f"c0 = {c0:.4f}")

Using these coefficients solve the differential equations to obtain a new approximation

In [ ]:
# define functions
# NOTE: you don't need to write docstrings for every function (though you could), but write enough so that it is clear how to use your functions
set = 2
def derivatives_a0a1c0(t, y, a0, a1, c0):
    R = 8.314
    cf, cag, c = y
    
    dcfdt = -(a0/np.exp(a1/T[set]))*(cf - ceq)
    dcagdt = (ca0[set]*(.00089)*((R*T[set])/(mu[set]))*c0)*c**2
    dcdt = (a0/np.exp(a1/T[set]))*(cf - ceq) - (ca0[set]*(.00089)*((R*T[set])/(mu[set]))*c0)*c**2
    
    return [dcfdt, dcagdt, dcdt]

# time span
t_span = (0.0, t_exp[-1])

# initial conditions
y0 = [1.0, 0.0, 0.0]

# Define a function that returns the model values for given parameters
def model(t, a0, a1, c0):
    sol = solve_ivp(fun=lambda t, y: derivatives_a0a1c0(t, y, a0, a1, c0), t_span=t_span, y0=y0, t_eval=t_exp)
    c_model = sol.y[0]
    return c_model

Plot the new function given by coefficients a0, a1 and c0

In [ ]:
# Plot the model predictions using the best coefficients
sol_best = solve_ivp(fun=lambda t, y: derivatives_a0a1c0(t, y, a0, a1, c0), t_span=t_span, y0=y0, t_eval=t_exp)
plt.plot(t_exp, c_exp, label='exp')
#plt.plot(sol_best.t, sol_best.y[0], label='cf')
plt.plot(sol_best.t, sol_best.y[1], label='cag')
#plt.plot(sol_best.t, sol_best.y[2], label='c')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.show()

#Show R2
print('R2 Score:',r2_score(c_exp,sol_best.y[1])*100,'%')